## Wandb
> Wandb adalah library untuk mencatat log histori ketika melakukan proses training. Pertama-tama, anda harus membuat akun dari situs [Wandb](https://wandb.ai/site).

Contoh tampilan dari *dashboard* wandb:

<div align="center">
<img src="../assets/wandb.png" width="400"/>
</div>

Dan pada program notebook ini, kita akan mencoba menggunakan model transfer learning dari vgg11 untuk melakukan training dengan dataset CIFAR10

### Menginstall Package Wandb
Anda dapat menginstall package python Wandb dengan cara di bawah ini:

Menggunakan pip:

```bash
pip install wandb
```

Menggunakan conda:

```bash
conda install wandb -c conda-forge
```

### Mengimpor Library Wandb dan Menginisialisasi


In [ ]:
import wandb

In [ ]:
wandb.init(project="vggcifarm1") # konfigurasi nama proyek (akan ditampilkan pada laman web)

> **Note**
> Ketika anda pertama kali menggunakan wandb, anda akan diminta untuk menginputkan API key yang didapat dari profil wandb anda. Anda dapat merujuk pada [tautan ini](https://wandb.ai/quickstart/pytorch) untuk referensi cepat wandb untuk framework PyTorch.

### Mengimpor Library Lainnya

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time

### Memilih Device

> **Note**
> Kode dibawah ini akan memilih device "cuda" apabila versi PyTorch mendukung komputasi dengan kartu grafis (GPU) nvidia. Jika tidak, maka akan memilih device "cpu".

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Memuat Model
Pada kode di bawah ini, kita akan memuat model dari vgg11 ke sebuah variable bernama vgg. Kita akan menggunakan model pretrained sehingga argumen `pretrained` akan diisi dengan `True`.

In [ ]:
vgg = torchvision.models.vgg11(pretrained=True)

Kita perlu untuk melakukan transfer learning dari model vgg11 karena dataset yang kita gunakan (CIFAR10) hanya memiliki 10 kelas.

Kita akan mengambil features layer dari vgg11 dan menambahkan fully-connected layer di bagian akhir dari model.

In [ ]:
features = vgg.features
flat_features = nn.Flatten() # layer untuk mengubah dimensi input menjadi (batch x dimensi) (dalam kasus ini mengubah dari data 2D menjadi 1D)
fc = nn.Linear(in_features=512, out_features=10)
model = nn.Sequential(features, flat_features, fc)
model = model.to(device)

### Menentukan Loss Function dan Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### Konfigurasi Hyperparameter

Anda dapat bereksperimen dengan hyperparameter berikut

In [ ]:
num_epoch = 4
batch_size = 32
learning_rate = 0.001

### Dataset dan Dataloader

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

### Training Iteration

In [ ]:
# Menghitung panjang dari trainloader
n_total_steps = len(trainloader)

print("Start Training...")

# Menghitung waktu mulai
start_time_train = time.time()


for epoch in range(num_epoch):
    start_time_epoch = time.time()
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(10)]
        n_class_samples = [0 for i in range(10)]

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for ii in range(labels.size(0)):
            label = labels[ii]
            pred = predicted[ii]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
        # Mengirim log ke wandb berupa loss dan accuracy
        wandb.log({'Training Loss': loss.item(), 'Training Accuracy': n_correct/n_samples*100})
        
        # Mencetak epoch dan loss setiap 50 step
        if (i+1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
        
    print(f'Time taken for epoch {epoch+1} is {time.time() - start_time_epoch}')
    
    # Mengirim log ke wandb berupa waktu yang dibutuhkan untuk setiap epoch
    wandb.log({'Time taken for epoch': time.time() - start_time_epoch})
    

print(f'Total time taken for training is {time.time() - start_time_train}')